## Factorization Machine
----

### Concept
- Factorization Machine is general predictor like SVM. FM calculate all of pair-wise interaction between variables. So, FM can overcome the situation `cold-start` because of pair-wise vector factorization. It works like latent vector, but break the independence of the interaction parameter by factorizating them. Therefore, this algorithm can have a similar effect to using SVD++ with multiple variables.
- This Algorithm works well especially in recommender system like e-commerce. Because of the sparsity in implicit data and content meta-information.
- [Paper is here](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)

### Model Equation
- Conceptional model equation

$$ \hat{y}(x) = w_0 + \sum_{i=1}^{n}w_i x_i + \sum_{i=1}^{n}\sum_{j=i+1}^{n} <v_i, v_j> x_i x_j $$

$$ <v_i, v_j> = \hat{w}_{ij} $$

- FM have a closed model equation that can be computed in linear time O(kn), but actually O(kmd) because of zero-values.  

$$ 0.5\sum_{i=1}^{n}\sum_{j=1}^{n} <v_i, v_j> x_i x_j - 0.5\sum_{i=1}^{n}<v_i, v_i> x_i x_i $$

$$ 0.5(\sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{f=1}^{k} v_{i,f} v_{j,f} x_i x_j - \sum_{i=1}^{n}\sum_{f=1}^{k} v_{i,f} v_{i,f} x_i x_i) $$

$$ 0.5\sum_{f=1}^{k}((\sum_{i=1}^{n} v_i x_i)(\sum_{j=1}^{n} v_j x_j) - \sum_{i=1}^{n} {v_i}^2{x_i}^2) $$

$$ 0.5\sum_{f=1}^{k}((\sum_{i=1}^{n} v_i x_i)^2 - \sum_{i=1}^{n} {v_i}^2{x_i}^2) $$

- Below code is implementation of FM's equation

In [78]:
import numpy as np

# Pre-trained parameters
b = 0.3
w = np.array([0.001, 0.02, 0.009, -0.001])
v = np.array([[0.00516, 0.0212581, 0.150338, 0.22903],
              [0.241989, 0.0474224, 0.128744, 0.0995021], 
              [0.0657265, 0.1858, 0.0223, 0.140097], 
              [0.145557, 0.202392, 0.14798, 0.127928]])

# Equation of FM model
def inference(data):
    num_data = len(data)
    scores = np.zeros(num_data)
    for n in range(num_data):
        feat_idx = data[n][0]
        val = np.array(data[n][1])
        
        # linear feature score
        linear_feature_score = np.sum(w[feat_idx] * val)
        
        # factorized feature score
        vx = v[feat_idx] * (val.reshape(-1, 1))
        cross_sum = np.sum(vx, axis=0)
        square_sum = np.sum(vx*vx, axis=0)
        cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)
        
        # Model's equation
        scores[n] = b + linear_feature_score + cross_feature_score

    # Sigmoid transformation for binary classification
    scores = 1.0 / (1.0 + np.exp(-scores))
    return scores

In [ ]:
# Inference test for 3 case
data = [[[0, 1, 3], # feature index 
         [0.33, 1, 1]], # feature value
        [[2],
         [1]],
        [[0, 1, 2, 3],
         [0.96, 1, 1, 1]]]

inference(data)

### Learning FM
- The equation computed in linear time. So we can use SGD with below gradients. And in most cases add L2 Regularization on training model.

$$ 1, \hspace{1cm} if \hspace{0.3cm} \theta \hspace{0.2cm} is \hspace{0.2cm} w_0 $$

$$ x_i, \hspace{1cm} if \hspace{0.3cm} \theta \hspace{0.2cm} is \hspace{0.2cm} w_i $$
 
$$ x_i \sum_{j=1}^{n} v_{j,f}x_{j} - v_{i,f} {x_i}^2 \hspace{1cm} if \hspace{0.3cm} \theta \hspace{0.2cm} is \hspace{0.2cm} v_{i,f} $$  

In [163]:
v = np.array([[10, 20],
              [30, 40],
              [50, 60],
              [70, 80]])
feat_idx = np.array([1,3]) ## x가 있는 곳의 위치
val = np.array([2,2]) ## x값
vx = v[feat_idx] * (val.reshape(-1, 1))

In [164]:
v[feat_idx] = v[feat_idx] - 0.001 * ((sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1)))) + 0.01 * v[feat_idx])

In [165]:
v

array([[10, 20],
       [29, 39],
       [50, 60],
       [69, 79]])

In [167]:
(sum(vx) * (val.reshape(-1, 1)))

array([[400, 480],
       [400, 480]])

In [169]:
(vx * (val.reshape(-1, 1)))

array([[120, 160],
       [280, 320]])

In [170]:
0.01 * v[feat_idx]

array([[0.29, 0.39],
       [0.69, 0.79]])

In [176]:
0.001*((sum(vx) * (val.reshape(-1, 1)))-(vx * (val.reshape(-1, 1)))+0.01 * v[feat_idx])

array([[0.28029, 0.32039],
       [0.12069, 0.16079]])

In [182]:
1.0 / (1.0 + np.exp(10))

4.5397868702434395e-05